# Script 1: ETL process

1) Extract a video information and statistic from its video's ID
2) Extract all of the comments from the video
3) Convert the dtype of video & comments dataframe

In [1]:
# import libraries
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd
import numpy as np

In [2]:
# get your own API_key
yt_api_key = xxx;

# access youtube data with API_key
api_service_name = "youtube"
api_version = "v3"
youtube = build(
    api_service_name, api_version, developerKey=yt_api_key)

In [3]:
# create a function to return the video statistic
def video_stats(youtube,video_id):
    request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id=",".join(video_id)
    )
    response = request.execute()

    video_data = []
    for i in range(len(response['items'])):
        data = dict(
            channelID=response['items'][i]['snippet']['channelId'],
            channelTitle=response['items'][i]['snippet']['channelTitle'],
            videoTitle=response['items'][i]['snippet']['title'],
            videoPublish=response['items'][i]['snippet']['publishedAt'],
            videoDuration=response['items'][i]['contentDetails']['duration'],
            viewCount=response['items'][i]['statistics']['viewCount'],
            likeCount=response['items'][i]['statistics']['likeCount'],
            commentCount=response['items'][i]['statistics']['commentCount'],
            )
        video_data.append(data)
    return(pd.DataFrame(video_data))

In [4]:
# create a function to get the comment information
def load_comment(response):
    data_comment=[]
    for i in range(len(response['items'])):
        comments = dict(
            commentID=response['items'][i]['snippet']['topLevelComment']['id'],
            commentPublish=response['items'][i]['snippet']['topLevelComment']['snippet']['publishedAt'],
            comment=response['items'][i]['snippet']['topLevelComment']['snippet']['textDisplay'],
            likeCount=response['items'][i]['snippet']['topLevelComment']['snippet']['likeCount'],
        )
        #comments_in_video_info = {'video_id': video_id, 'comments': comments}
        data_comment.append(comments)
    return data_comment

In [5]:
# create a function to extract the comment from API

def get_comments(youtube, video_id):
    for video in video_id:
        initial_request = youtube.commentThreads().list(
            part='snippet',
            videoId=video,
            maxResults=100
        )
        initial_response = initial_request.execute()

        a = load_comment(initial_response)  # Assuming this function appends comments to list a

        # Loop through subsequent requests
        while 'nextPageToken' in initial_response:
            next_page_token = initial_response['nextPageToken']
        
            next_request = youtube.commentThreads().list(
                part='snippet',
                videoId=video,
                maxResults=100,
                pageToken=next_page_token
            )
        
            next_response = next_request.execute()

            a += load_comment(next_response)
        
            # Update initial_response for the next iteration
            initial_response = next_response

    # Assuming load_comment correctly structures comments in a list
    # Convert the list of comments to a pandas DataFrame
    return pd.DataFrame(a)

In [6]:
# the IDs of video I want to do sentiment analysis
video_id = [
        "U2v76H_B1rs",
        ]

In [7]:
# stored the video statistic into a dataframe
df = video_stats(youtube,video_id)

In [8]:
# transform the video dataframe
# convert to numerical dtypes
numerical_col= ['viewCount','likeCount','commentCount']
df[numerical_col] = df[numerical_col].apply(pd.to_numeric, axis= 1)

In [9]:
# convert to datetime dtype
from datetime import datetime
df['videoPublish'] = df['videoPublish'].apply(pd.to_datetime, errors='coerce')

In [10]:
# convert duration column into seconds
from isodate import parse_duration
df["videoDuration"] = df["videoDuration"].astype('timedelta64[s]')
df["videoDuration"]= df["videoDuration"].apply(lambda x: x.total_seconds())

In [11]:
# show the video dataframe
df

,channelID,channelTitle,videoTitle,videoPublish,videoDuration,viewCount,likeCount,commentCount
0,UCJQJAI7IjbLcpsjWdSzYz0Q,Thu Vu data analytics,What Data Science Courses DON'T TEACH YOU 🤫,2022-02-06 18:21:13+00:00,784.0,159585,7840,324


In [12]:
# stored the comment into a dataframe
df_comment = get_comments(youtube,video_id)

In [13]:
# transform the comment dataframe
# convert to datetime dtype
df_comment['commentPublish'] = df_comment['commentPublish'].apply(pd.to_datetime, errors='coerce')

In [15]:
# show the comment dataframe
df_comment

,commentID,commentPublish,comment,likeCount
0,Ugxs63tEl4SKQaUEc3p4AaABAg,2022-03-26 11:20:29+00:00,I agree with every word you have said and I&#3...,99
1,UgyJyF4NZ9Ln_GexbXF4AaABAg,2023-07-19 22:04:58+00:00,Hi Thu. Just coming across your video that is ...,0
2,Ugx9rxnuxZ8AMhkGPnV4AaABAg,2023-06-11 02:07:23+00:00,yes it is confirmed - gender bias is nowadays ...,0
3,UgzIyLHLWR6rs0JBiLt4AaABAg,2023-06-05 23:01:23+00:00,please do write the ebook,0
4,UgzP7Qdx6cpF71wBSbZ4AaABAg,2023-05-07 07:47:01+00:00,This is what happens when people use computers...,0
...,...,...,...,...
208,UgzMcsF9N4-MNyW6Ihh4AaABAg,2022-02-07 02:10:12+00:00,Where do you get such format like you have in ...,2
209,UgwoTSYqyFzRizrDqT14AaABAg,2022-02-07 02:10:06+00:00,Where do you get such format like you have in ...,2
210,Ugw0aVikxhgi6vNmg8d4AaABAg,2022-02-06 22:31:36+00:00,Loved the skits throughout this! 🙌🏼😂<br>Also t...,31
211,UgwHfm1sxPnrnFyO79Z4AaABAg,2022-02-06 19:01:24+00:00,I am a big fan of you. 😁😁😁 Hope you can talk a...,1
